In [2]:
# using genetic algorithms to optimize simple machine learning models, such 
# as the random forest classifier, to implement brain tumor classification
# This files does not implement tpot and only uses one specific version of a genetic algorithm
# The dataset used for this model is images of brain tumors available on Kaggle
# https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset
# https://medium.com/@ela.markovic/feature-selection-using-genetic-algorithm-complete-beginner-friendly-guide-198496393728
# https://www.geeksforgeeks.org/random-forest-for-image-classification-using-opencv/


In [ ]:
# import libraries
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
import cv2
import os

In [41]:
def load_data(directory, img_size=(64, 64)):
    data = []
    labels = []
    class_names = []
    for label in os.listdir(directory):
        label_dir = os.path.join(directory, label)
        if not os.path.isdir(label_dir):
            continue
        class_names.append(label)
        for file in os.listdir(label_dir):
            file_path = os.path.join(label_dir, file)
            if file.lower().endswith(('.jpg', '.png', '.jpeg')):
                img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE)
                if img is not None:
                    img = cv2.resize(img, img_size)
                    img_flat = img.flatten()
                    data.append(img_flat)
                    labels.append(label)
    return np.array(data), np.array(labels), class_names

In [44]:
# Paths to your folders
base_dir = os.path.expanduser("brain_tumor_mri") 
train_dir = os.path.join(base_dir, "Training")
test_dir = os.path.join(base_dir, "Testing")

In [45]:
# Load images
X_train, y_train, class_names = load_data(train_dir)
X_test, y_test, _ = load_data(test_dir)


In [46]:
# Encode labels numerically
le = LabelEncoder()
y_train_enc = le.fit_transform(y_train)
y_test_enc = le.transform(y_test)

In [53]:
# Hyperparameter ranges
N_ESTIMATORS_MIN = 50
N_ESTIMATORS_MAX = 200
MAX_DEPTH_MIN = 1
MAX_DEPTH_MAX = 50

# Genetic Algorithm parameters
POP_SIZE = 100
GENERATIONS = 10
MUTATION_RATE = 0.1
CROSSOVER_RATE = 0.5
TOURNAMENT_SIZES = 5

In [54]:
# Initialize random population
population = [ 
    [np.random.randint(N_ESTIMATORS_MIN, N_ESTIMATORS_MAX + 1), 
     np.random.randint(MAX_DEPTH_MIN, MAX_DEPTH_MAX + 1)] 
    for _ in range(POP_SIZE) 
]

In [55]:
def eval_genome(genome):
    n_estimators = int(genome[0])
    max_depth = int(genome[1])
    clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    clf.fit(X_train, y_train_enc)
    y_pred = clf.predict(X_test)
    acc = accuracy_score(y_test_enc, y_pred)
    return acc

def tournament_selection(pop, fitnesses):
    selected = []
    for _ in range(POP_SIZE):
        competitors_idx = np.random.choice(len(pop), TOURNAMENT_SIZES, replace=False)
        best_idx = competitors_idx[np.argmax([fitnesses[i] for i in competitors_idx])]
        selected.append(pop[best_idx].copy())
    return selected

def crossover(parent1, parent2):
    if np.random.rand() < CROSSOVER_RATE:
        point = np.random.randint(1, len(parent1))
        child1 = parent1[:point] + parent2[point:]
        child2 = parent2[:point] + parent1[point:]
        return child1, child2
    else:
        return parent1.copy(), parent2.copy()

def mutate(genome):
    if np.random.rand() < MUTATION_RATE:
        gene = np.random.choice([0,1])
        if gene == 0:
            genome[0] = np.random.randint(N_ESTIMATORS_MIN, N_ESTIMATORS_MAX + 1)
        else:
            genome[1] = np.random.randint(MAX_DEPTH_MIN, MAX_DEPTH_MAX + 1)
    return genome

In [56]:
# GA main loop
best_accuracies = []  # To track best accuracy over generations

for gen in range(GENERATIONS):
    print(f"Evaluating Generation {gen + 1}")
    fitnesses = [eval_genome(g) for g in population]
    best_accuracy = np.max(fitnesses)
    avg_accuracy = np.mean(fitnesses)
    best_accuracies.append(best_accuracy)
    
    print(f"Generation {gen}: Best Accuracy = {best_accuracy:.4f}, Avg Accuracy = {avg_accuracy:.4f}")

    # Selection
    selected_population = tournament_selection(population, fitnesses)

    # Crossover and Mutation
    next_population = []
    for i in range(0, POP_SIZE, 2):
        parent1 = selected_population[i]
        parent2 = selected_population[(i+1) % POP_SIZE]
        child1, child2 = crossover(parent1, parent2)
        child1 = mutate(child1)
        child2 = mutate(child2)
        next_population.extend([child1, child2])

    # Update the population
    population = next_population[:POP_SIZE]

Evaluating Generation 1
Generation 0: Best Accuracy = 0.9382, Avg Accuracy = 0.8890
Evaluating Generation 2
Generation 1: Best Accuracy = 0.9390, Avg Accuracy = 0.9355
Evaluating Generation 3
Generation 2: Best Accuracy = 0.9390, Avg Accuracy = 0.9333
Evaluating Generation 4
Generation 3: Best Accuracy = 0.9390, Avg Accuracy = 0.9354
Evaluating Generation 5
Generation 4: Best Accuracy = 0.9390, Avg Accuracy = 0.9342
Evaluating Generation 6
Generation 5: Best Accuracy = 0.9397, Avg Accuracy = 0.9356
Evaluating Generation 7
Generation 6: Best Accuracy = 0.9397, Avg Accuracy = 0.9388
Evaluating Generation 8
Generation 7: Best Accuracy = 0.9397, Avg Accuracy = 0.9388
Evaluating Generation 9
Generation 8: Best Accuracy = 0.9397, Avg Accuracy = 0.9392
Evaluating Generation 10
Generation 9: Best Accuracy = 0.9397, Avg Accuracy = 0.9355


In [ ]:
# Final evaluation
final_fitnesses = [eval_genome(g) for g in population]
best_idx = np.argmax(final_fitnesses)
best_genome = population[best_idx]

print("\nBest hyperparameters found:")
print(f"n_estimators: {best_genome[0]}, max_depth: {best_genome[1]}")
print(f"Test accuracy: {final_fitnesses[best_idx]:.4f}")

# Train final model
best_model = RandomForestClassifier(
    n_estimators=int(best_genome[0]),
    max_depth=int(best_genome[1]),
    random_state=42
)
best_model.fit(X_train, y_train_enc)
y_pred_final = best_model.predict(X_test)

print("\nFinal Model Evaluation:")
print(f"Accuracy: {accuracy_score(y_test_enc, y_pred_final):.4f}")

# (Optional) Plotting Accuracy Progress Over Generations
import matplotlib.pyplot as plt
plt.plot(best_accuracies)
plt.xlabel('Generation')
plt.ylabel('Best Accuracy')
plt.title('Best Accuracy Over Generations')
plt.grid(True)
plt.show()